In [1]:
# import libraries

from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS, ALSModel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from cold_start import get_cold_start_rating


In [ ]:
# make a SparkSession object

spark = (SparkSession
         .builder
         .appName("MoviesALS")
         .config("spark.driver.host", "localhost")
         .getOrCreate())

In [ ]:
# import ratings json file into spark dataframe

movie_ratings = spark.read.json('data/ratings.json')

In [ ]:
# check schema
movie_ratings.printSchema()

In [ ]:
# cast to Pandas dataframe to turn timestamp data to datetime and check nulls. 

movies_df = movie_ratings.select('*').toPandas()
movies_df.info()

In [ ]:
# attempt to change timestamp object to years, all years are 2000

date = pd.to_datetime(movies_df['timestamp'], unit='s').dt.year
date.value_counts()

In [ ]:
# Decide to drop timestamp for now because only year 2000

movie_ratings = movie_ratings.drop('timestamp')

In [ ]:
# Split data into training and test set

(training, test) = movie_ratings.randomSplit([.8, .2])

In [ ]:
# Create ALS instance and fit model

als = ALS(maxIter=10,
          rank=10,
          userCol='user_id',
          itemCol='movie_id',
          ratingCol='rating',
          seed=42)

model = als.fit(training)

In [ ]:
# Generate Predictions

predictions = model.transform(test)
predictions.persist()

In [ ]:
# Convert to pandas dataframe, fill prediction nulls, and convert back to spark dataframe

pred_df = predictions.select('*').toPandas()

In [ ]:
pred_df.isna().sum()

In [ ]:
def user_average(user, df):
    """Return average score for user"""
    user_df = df[df['user_id'] == user]
    average = user_df['prediction'].mean()
    if np.isnan(average):
        return 3
    else:
        return average
    
def compute_user_average_if_null(row):
    """Check if value is null, if so, replace with user average"""
    if np.isnan(row['prediction']):
        return user_average(row['user_id'], pred_df)
    else:
        return row['prediction']
    

In [ ]:
for i, row in pred_df[pred_df['prediction'].isna()].iterrows():
    pred_df.loc[i, 'prediction'] = get_cold_start_rating(row['user_id'], row['movie_id'])
    
print(pred_df['prediction'].isna().any())
    

In [ ]:
predictions = spark.createDataFrame(pred_df)

In [ ]:
# Evaluate model 
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating',
                               predictionCol='prediction')

rmse = evaluator.evaluate(predictions)
print(rmse)

In [ ]:
# Create a parameter grid, cross validate for best model with different hyperperameters
params_score = {}

params = (ParamGridBuilder()
          .addGrid(als.regParam, [1, 0.01, 0.001, 0.1])
          .addGrid(als.maxIter, [5, 10, 20])
          .addGrid(als.rank, [4, 10, 50])).build()

cv = CrossValidator(estimator=als, estimatorParamMaps=params, evaluator=evaluator, parallelism=4)

best_model = cv.fit(movie_ratings)
als_model = best_model.bestModel

In [ ]:
# save model
als_model.save('als_model')

In [ ]:
# load requests json file into a spark dataframe
    
requests = spark.read.json("data/requests.json") 

# predict requests with als model
requests_predictions = model.transform(requests).toPandas()

# predict null predictions with cold start model
for i, row in requests_predictions[requests_predictions['prediction'].isna()].iterrows():
    requests_predictions.loc[i, 'prediction'] = get_cold_start_rating(row['user_id'], row['movie_id'])
    
print(requests_predictions['prediction'].isna().any())

# export request predictions dataframe as json file.
predictions = requests_predictions.to_json(r"data/predictions.json")